In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
import os
import random
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity


## Reading dataset

In [9]:
if not os.path.isfile("./data/NetflixRating.csv"):
    startTime=datetime.now()
    data=open("./Data/NetflixRatings.csv",mode="w")
    
    files=["./Data/combined_data_2.txt",'./Data/combined_data_4.txt']
    for file in files:
        print("Reading from file: "+str(file)+"...")
        with open(file) as f:
            
            for line in f:
                line=line.strip()
                
                if line.endswith(":"):
                    movieID=line.replace(":","")
                else:
                    row=[]
                    row=[x for x in line.split(",")]
                    row.insert(0,movieID)
                    data.write(",".join(row))
                    data.write("\n")
        print("Reading of file"+str(file)+" is completed\n")
    data.close()
    print("Total time taken" + str(datatime.now()-startTime))
    

Reading from file: ./Data/combined_data_2.txt...
Reading of file./Data/combined_data_2.txt is completed

Reading from file: ./Data/combined_data_4.txt...
Reading of file./Data/combined_data_4.txt is completed



NameError: name 'datatime' is not defined

In [ ]:
if not os.path.isfile("./Data/NetflixData.pkl"):
    startTime=datetime.now()